In [15]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, roc_auc_score
from pyod.models.iforest import IForest

from itertools import product


import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from sklearn import __version__ as sklearn_version
print(pd.__version__)
print(sklearn_version)

import h5py

1.0.5
0.23.1


In [24]:
class UFIF():
    """
    Universal Filled Isolation Forest.
    Wrapper of the normal one.
    """
    contamination=0.1
    
    X=None
    X_fill=None
    X_filled=None
    
    iforest:IForest=None
    
    def __init__(self, n_estimators=100, contamination=0.1, random_state=None,):
        self.iforest=IForest(n_estimators=n_estimators, contamination=contamination, random_state=random_state)
        
    def get_filling(self, n:int):
        return np.array(list(product([0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1], repeat=n)))
        
    def fit(self, X, y=None):
        n_features = X.shape[1]
        X_fill = self.get_filling(n_features)
        X_filled = np.concatenate([X, X_fill], axis=0)
        self.iforest.fit(X_filled)
        return self
    
    def predict_proba(self, X):
        return self.iforest.predict_proba(X)

In [94]:
def get_X_y_73(filename):
    f=h5py.File(filename, 'r')
    d={}
    for k, v in f.items():
        d[k]=v
        print(k, v)
    X=d['X'].value.T
    y=d['y'].value[0]
    
    X=X-np.min(X, axis=0)
    X=X/np.max(X, axis=0)
    
    return X, y

def get_X_y_50(filename):
    dat = sio.loadmat(filename)
    X=dat['X']
    y=dat['y']
    X=X-np.min(X, axis=0)
    X=X/np.max(X, axis=0)
    
    return X, y

def compare(X, y):
    # X, y = get_X_y(filename)
    n_features = X.shape[1]
    n_samples = len(y)
    n_anomalies = sum(y)
    print(f"n_samples:{n_samples}, n_anomalies:{n_anomalies}")
    contamination=n_anomalies/n_samples

    if_aucs=[]
    ufif_aucs=[]
    for random_state in [11,22,33,44,55,66,77,88,99,101]:
        print(f'random_state {random_state}')
        clf = IForest(contamination=contamination, random_state=101)
        clf.fit(X)
        y_proba=clf.predict_proba(X)
        if_auc=roc_auc_score(y ,y_proba[:,1])
        if_aucs.append(if_auc)

        ufif = UFIF(contamination=contamination, random_state=101)
        ufif.fit(X)
        y_proba=ufif.predict_proba(X)
        ufif_auc=roc_auc_score(y ,y_proba[:,1])
        ufif_aucs.append(ufif_auc)
        
        print(if_auc, ufif_auc)

    if_auc = np.mean(if_aucs)
    ufif_auc = np.mean(ufif_aucs)
    return n_samples, n_features, n_anomalies, if_auc, ufif_auc

In [52]:
a, b=compare('data/smtp.mat')

X <HDF5 dataset "X": shape (3, 95156), type "<f8">
y <HDF5 dataset "y": shape (1, 95156), type "<f8">


<ipython-input-4-5510a7f64767>:7: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  X=d['X'].value.T
<ipython-input-4-5510a7f64767>:8: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  y=d['y'].value[0]


In [72]:
ha, hb=compare('data/http.mat')
ha, hb

X <HDF5 dataset "X": shape (3, 567498), type "<f8">
y <HDF5 dataset "y": shape (1, 567498), type "<f8">
random_state 11


<ipython-input-66-6e0d28853414>:7: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  X=d['X'].value.T
<ipython-input-66-6e0d28853414>:8: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  y=d['y'].value[0]


random_state 22
random_state 33
random_state 44
random_state 55
random_state 66
random_state 77
random_state 88
random_state 99
random_state 101


(0.9998397567140154, 0.9995045979761865)

In [95]:
X,y=get_X_y_50('data/pima.mat')
compare(X, y)

random_state 11


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_iforest.py:304: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self.contamination == "auto":


MemoryError: Unable to allocate 6.39 GiB for an array with shape (214359649, 8) and data type float32

In [62]:
a, b

(0.9020227557835572, 0.9085901856485084)

In [ ]:
X, y = get_X_y_50('data/cover.mat')
ca, cb=compare(X, y)

In [82]:
import scipy.io as sio

dat = sio.loadmat('data/pima.mat')

In [80]:
dat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Tue Apr 26 19:33:16 2016',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
        [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
        [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
        ...,
        [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
        [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
        [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]]),
 'y': array([[1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
       

In [89]:
    dat = sio.loadmat('data/pima.mat')
    X=dat['X']
    y=dat['y']
    
    X=X-np.min(X, axis=0)
    X=X/np.max(X, axis=0)
    


In [91]:
X.shape

(768, 8)